In [1]:
pip install pycocoevalcap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 MB 16.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pycocoevalcap.bleu.bleu import Bleu
from pycocoevalcap.meteor.meteor import Meteor
from pycocoevalcap.rouge.rouge import Rouge
from pycocoevalcap.cider.cider import Cider
from pycocoevalcap.spice.spice import Spice

In [3]:
def evaluate_metrics(references, hypotheses):
    # Initializing scorers
    bleu_scorer = Bleu(4)
    meteor_scorer = Meteor()
    rouge_scorer = Rouge()
    cider_scorer = Cider()
    spice_scorer = Spice()

    # Compute scores
    bleu_score, _ = bleu_scorer.compute_score(references, hypotheses)
    meteor_score, _ = meteor_scorer.compute_score(references, hypotheses)
    rouge_score, _ = rouge_scorer.compute_score(references, hypotheses)
    cider_score, _ = cider_scorer.compute_score(references, hypotheses)
    spice_score, _ = spice_scorer.compute_score(references, hypotheses)

    # Combine all scores
    scores = {
        'BLEU-1': bleu_score[0],
        'BLEU-2': bleu_score[1],
        'BLEU-3': bleu_score[2],
        'BLEU-4': bleu_score[3],
        'METEOR': meteor_score,
        'ROUGE_L': rouge_score,
        'CIDEr': cider_score,
        'SPICE': spice_score,
    }

    return scores

In [ ]:
import torch

# This evaluation is for the BLIP model that has been fine-tuned using only 7.000 data samples (BLIP-Base version)
model_path = '/kaggle/input/base-7000-blip/blip-ecg-capt-base7000.pth' 
#state_dict = torch.load(model_path, map_location=torch.device('cpu'))
state_dict = torch.load(model_path)

# Print the keys in the state dictionary
print(state_dict.keys())

odict_keys(['vision_model.embeddings.class_embedding', 'vision_model.embeddings.position_embedding', 'vision_model.embeddings.patch_embedding.weight', 'vision_model.embeddings.patch_embedding.bias', 'vision_model.encoder.layers.0.self_attn.qkv.weight', 'vision_model.encoder.layers.0.self_attn.qkv.bias', 'vision_model.encoder.layers.0.self_attn.projection.weight', 'vision_model.encoder.layers.0.self_attn.projection.bias', 'vision_model.encoder.layers.0.layer_norm1.weight', 'vision_model.encoder.layers.0.layer_norm1.bias', 'vision_model.encoder.layers.0.mlp.fc1.weight', 'vision_model.encoder.layers.0.mlp.fc1.bias', 'vision_model.encoder.layers.0.mlp.fc2.weight', 'vision_model.encoder.layers.0.mlp.fc2.bias', 'vision_model.encoder.layers.0.layer_norm2.weight', 'vision_model.encoder.layers.0.layer_norm2.bias', 'vision_model.encoder.layers.1.self_attn.qkv.weight', 'vision_model.encoder.layers.1.self_attn.qkv.bias', 'vision_model.encoder.layers.1.self_attn.projection.weight', 'vision_model.en

In [5]:
from collections import OrderedDict
from transformers import BlipForConditionalGeneration, BlipProcessor

model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# Remove unexpected keys
def remove_unexpected_keys(state_dict):
    expected_keys = model.state_dict().keys()
    new_state_dict = OrderedDict()
    for key, value in state_dict.items():
        if key in expected_keys:
            new_state_dict[key] = value
    return new_state_dict

# Adjust the state dictionary
adjusted_state_dict = remove_unexpected_keys(state_dict)

2025-05-31 15:06:21.597316: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748703981.799447      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748703981.861866      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [6]:
#from transformers import BlipForConditionalGeneration, BlipProcessor

# Load the model and processor
#model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
adjusted_state_dict = remove_unexpected_keys(state_dict)
model.load_state_dict(adjusted_state_dict)
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
#processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base", map_location=torch.device('cpu'))

model.eval()  # Set the model to evaluation mode

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

BlipForConditionalGeneration(
  (vision_model): BlipVisionModel(
    (embeddings): BlipVisionEmbeddings(
      (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (encoder): BlipEncoder(
      (layers): ModuleList(
        (0-11): 12 x BlipEncoderLayer(
          (self_attn): BlipAttention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (projection): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): BlipMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((768,), eps=1e-0

In [7]:
import pandas as pd

# Load the test dataset
ref_path = '/kaggle/input/base-7000-blip/Test_Dataset_BLIP-ECG-Base-7000datas.csv'
test_data = pd.read_csv(ref_path)

# Assuming the CSV has columns 'image_path' and 'caption'
image_paths = test_data['image'].tolist()
reference_captions = test_data['caption'].tolist()

In [8]:
import os

base_image_path = '/kaggle/input/ptb-xl-ecg-processed/PTB-XL ECG Dataset/PTB-XL ECG Dataset'

# Update the image paths
image_paths = [os.path.join(base_image_path, img_path) for img_path in image_paths]

In [9]:
from PIL import Image
from tqdm import tqdm

generated_captions = []

# Tambahkan tqdm agar terlihat progres saat loop
for image_path in tqdm(image_paths, desc="Generating captions"):
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt")
    outputs = model.generate(**inputs, max_new_tokens=50)
    caption = processor.decode(outputs[0], skip_special_tokens=True)
    generated_captions.append(caption)


Generating captions: 100%|██████████| 700/700 [32:33<00:00,  2.79s/it]


In [10]:
references = {i: [cap] for i, cap in enumerate(reference_captions)}
candidates = {i: [cap] for i, cap in enumerate(generated_captions)}

assert isinstance(references, dict)
assert isinstance(candidates, dict)

In [11]:
scores = evaluate_metrics(references, candidates)
df_scores = pd.DataFrame.from_dict(scores, orient='index', columns=['Score'])
print(df_scores)

Progress: 384.5M / 384.5M (100.0%)
Extracting stanford-corenlp-3.6.0 ...
Done.
{'testlen': 9359, 'reflen': 11835, 'guess': [9359, 8659, 7959, 7259], 'correct': [3663, 1939, 988, 623]}
ratio: 0.7907900295732326


Parsing reference captions
Initiating Stanford parsing pipeline
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.pipeline.TokenizerAnnotator - TokenizerAnnotator: No tokenizer type provided. Defaulting to PTBTokenizer.
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ssplit
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator parse
[main] INFO edu.stanford.nlp.parser.common.ParserGrammar - Loading parser from serialized file edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz ... 
done [0.6 sec].
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator lemma
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ner
Loading classifier from edu/stanford/nlp/models/ner/english.all.3class.distsim.crf.ser.gz ... done [1.5 sec].
Loading classifier from edu/stanford/nlp/models/ner/english.muc.7class.distsim.crf.ser.gz ... done [0.8 sec].
Loading classif

SPICE evaluation took: 1.408 min
            Score
BLEU-1   0.300408
BLEU-2   0.227228
BLEU-3   0.170076
BLEU-4   0.134172
METEOR   0.176055
ROUGE_L  0.346744
CIDEr    0.650180
SPICE    0.109952
